In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from textwrap import dedent
from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
with open("schemas/function.schema.json") as file:
    schema = json.load(file)

In [4]:
await toolkit.code.create_sandbox()

In [5]:
preface = dedent("""
    When generating code to access/interact with external systems for the user, please generate the
    code in Python only. Follow the below guidelines as well:

    - Do not generate examples that use the function.
    - Do not print the result, return it from the function.
    - Do not handle errors by printing or ignoring them, raise a custom exception with a user-friendly
      message instead.
""")

task = dedent("""
    I'd like to be able to ask you for the weather. Could you please write code to use the wttr.in API
    to fetch the weather for me when I need it? Just make a request to this URL:

            "http://wttr.in/<place>?format=(%l) %C and %t (feels %f) with winds at %w"
""")

In [6]:
messages = [{"role": "user", "content": preface}, {"role": "user", "content": task}]

In [7]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

```python
import requests

class WeatherError(Exception):
    """Custom exception for weather-related errors."""
    pass

def get_weather(place):
    """
    Fetches the weather for a given place using the wttr.in API.

    Args:
        place (str): The place to fetch the weather for.

    Returns:
        str: The weather information.

    Raises:
        WeatherError: If the request to the API fails.
    """
    url = f"http://wttr.in/{place}?format=(%l) %C and %t (feels %f) with winds at %w"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        raise WeatherError(f"Failed to fetch weather for {place}: {e}")
```

In [8]:
function = toolkit.functions.parse_funcs(complete_response)[0]

In [9]:
task = dedent(f"""
    Generate a function specification for the function you just generated. Make sure it is produced in a code block
    with the language set to `function_spec` instead of `json`. Make sure to follow the below schema:

    {json.dumps(schema, indent=2)}
""")

In [10]:
messages.append({"role": "user", "content": task})

In [11]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

```json
{
  "name": "get_weather",
  "description": "Fetches the weather for a given place using the wttr.in API. Returns a string containing the current weather conditions.",
  "parameters": {
    "type": "object",
    "properties": {
      "place": {
        "type": "string",
        "description": "The city or location to fetch the weather for."
      }
    },
    "required": [
      "place"
    ]
  },
  "responses": [
    {
      "type": "string",
      "description": "A string containing the weather information, formatted by the wttr.in API."
    }
  ],
  "errors": [
    {
      "name": "WeatherError",
      "description": "Raised if the request to the wttr.in API fails due to network issues or an invalid place name."
    }
  ],
  "examples": []
}
```

In [12]:
specification = toolkit.functions.parse_specs(complete_response)[0]

In [13]:
function = await toolkit.code.define_function(specification["name"], function)
toolkit.functions.register(function, specification)

In [14]:
prompt = toolkit.prompts.function_calling(model)
messages.append({"role": "user", "content": prompt})

In [15]:
task = "What's the weather in Pune and Singapore?"

In [16]:
messages.append({"role": "user", "content": task})

In [17]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
I need to fetch the weather for two locations: Pune and Singapore. I have a function `get_weather` that can fetch weather information for a given place. Since fetching weather for Pune and Singapore are independent tasks, I can call the function in parallel for both locations.

Guidelines followed:
- Identified the need for a function call.
- Identified that the tasks are independent and can be executed in parallel.
- Prepared two function calls for Pune and Singapore.

Guidelines not followed: None.
</blockquote>



```json
{
	"id": "pune_weather",
	"function": "get_weather",
	"parameters": {
		"place": "Pune"
	}
}
```

```json
{
	"id": "singapore_weather",
	"function": "get_weather",
	"parameters": {
		"place": "Singapore"
	}
}
```

In [18]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
render_markdown(outputs)

```json
{
    "id": "pune_weather",
    "result": "(Pune) Patchy rain nearby and +23\u00b0C (feels +25\u00b0C) with winds at \u219215km/h"
}
```

```json
{
    "id": "singapore_weather",
    "result": "(Singapore) Partly cloudy and +30\u00b0C (feels +36\u00b0C) with winds at \u219710km/h"
}
```


In [19]:
messages.append({"role": "user", "content": outputs})

In [20]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

The weather in Pune is patchy rain with a temperature of 23°C (feels like 25°C) and winds at 15km/h. The weather in Singapore is partly cloudy with a temperature of 30°C (feels like 36°C) and winds at 10km/h.

In [21]:
await toolkit.code.cleanup_sandbox()